In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [0]:
flight_df=spark.read.format("csv")\
    .option("header","true")\
    .option("inferSchema","true")\
    .load("/Volumes/workspace/default/files/2010-summary.csv")

In [0]:
flight_df.show()

+--------------------+-------------------+-----+
|   DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+--------------------+-------------------+-----+
|       United States|            Romania|    1|
|       United States|            Ireland|  264|
|       United States|              India|   69|
|               Egypt|      United States|   24|
|   Equatorial Guinea|      United States|    1|
|       United States|          Singapore|   25|
|       United States|            Grenada|   54|
|          Costa Rica|      United States|  477|
|             Senegal|      United States|   29|
|       United States|   Marshall Islands|   44|
|              Guyana|      United States|   17|
|       United States|       Sint Maarten|   53|
|               Malta|      United States|    1|
|             Bolivia|      United States|   46|
|            Anguilla|      United States|   21|
|Turks and Caicos ...|      United States|  136|
|       United States|        Afghanistan|    2|
|Saint Vincent and..

In [0]:
flight_df.count()

255

In [0]:
partition_flight_df=flight_df.repartition(4)

In [0]:
partition_flight_df.withColumn("partitionId",spark_partition_id()).groupBy("partitionId").count().show()

+-----------+-----+
|partitionId|count|
+-----------+-----+
|          0|   63|
|          1|   64|
|          2|   64|
|          3|   64|
+-----------+-----+



In [0]:
coalease_flight_df=flight_df.repartition(8)

In [0]:
coalease_flight_df.withColumn("partitionId",spark_partition_id()).groupBy("partitionId").count().show()

+-----------+-----+
|partitionId|count|
+-----------+-----+
|          0|   32|
|          1|   31|
|          2|   32|
|          3|   32|
|          4|   32|
|          5|   32|
|          6|   32|
|          7|   32|
+-----------+-----+



In [0]:
three_coalease_flight_df=coalease_flight_df.coalesce(3)

In [0]:
three_coalease_flight_df.withColumn("partitionId",spark_partition_id()).groupBy("partitionId").count().show()

+-----------+-----+
|partitionId|count|
+-----------+-----+
|          0|   63|
|          1|   96|
|          2|   96|
+-----------+-----+



① **What is repartitioning in Spark?**

- `repartition()` is used to increase or decrease the number of partitions in a DataFrame or RDD.
- It always involves a **full shuffle** of the data across the cluster.
- This ensures that data is **evenly distributed** across the specified partitions.

**Example:**  
python
df.repartition(10)  # creates 10 partitions with roughly equal data


👉 **Use case:**  
Use `repartition()` when you need better parallelism, such as before wide transformations or writing to files.

 ② What is coalesce in Spark?
 
 **coalesce()** is used to decrease the number of partitions in a DataFrame (cannot efficiently increase partitions).
 - It avoids a full shuffle by merging existing partitions.
 - Faster than repartition(), but may result in uneven data distribution.
 - Example: `df.coalesce(2)` reduces the DataFrame to 2 partitions by merging, with no shuffle.
 - **Use case:** When you want to reduce partitions, such as optimizing file writes with fewer output files.

③ **Which one will you choose and why?**

**Choose `repartition()` if:**
- You want to **increase** the number of partitions.
- You need a **balanced distribution** of data.
- You're preparing data for **parallel processing**.

**Choose `coalesce()` if:**
- You only want to **reduce** the number of partitions.
- **Performance** is more important than perfect balance.
- You want to **avoid a full shuffle** and **save time**.

🔥 Interview Tip:
If asked in an interview:

“If I want to reduce partitions before writing to HDFS, which is better?” → Coalesce.

“If I want evenly distributed partitions for parallelism?” → Repartition.